## import module

In [1]:
import os
import subprocess
import random
import warnings
import numpy as np
from scipy.linalg import sqrtm
from scipy.stats import unitary_group
import pandas as pd
import yaml
from tqdm.notebook import tqdm
import itertools

import torch
from qucumber.nn_states import DensityMatrix
from qucumber.callbacks import MetricEvaluator
import qucumber.utils.unitaries as unitaries
import qucumber.utils.training_statistics as ts
import qucumber.utils.cplx as cplx
import qucumber.utils.data as data
from qucumber.observables import ObservableBase, to_pm1
from qucumber.observables.pauli import flip_spin
import qucumber

from qulacs.gate import Pauli

%load_ext autoreload
%autoreload 2
import utils
import gate
import measurement
import target_circuit
import dataset

## setting params

In [2]:
with open('./params_setting.yaml', 'r') as yml:
    params = yaml.safe_load(yml)
    
# quantum circuit parameter
circuit_name = params["circuit_info"]["circuit_name"]
n_qubit = params["circuit_info"]["n_qubit"]
state_class = params["circuit_info"]["state_class"]
error_model = params["circuit_info"]["error_model"]
error_rate = params["circuit_info"]["error_rate"]
each_n_shot = params["circuit_info"]["each_n_shot"]

# RBM architecture parameter
num_visible = params["architecture_info"]["n_visible_unit"]
num_hidden = params["architecture_info"]["n_hidden_unit"] 
num_aux = params["architecture_info"]["n_aux_unit"]

# train parameter
lr = params["train_info"]["lr"]
pbs = params["train_info"]["positive_batch_size"]
nbs = params["train_info"]["negative_batch_size"]
n_gibbs_step = params["train_info"]["n_gibbs_step"]
period = 1
epoch = params["train_info"]["n_epoch"]
lr_drop_epoch = params["train_info"]["lr_drop_epoch"]
lr_drop_factor = params["train_info"]["lr_drop_factor"]
use_gpu = params["train_info"]["use_gpu"]
seed = params["train_info"]["seed"]

# sampling parameter
n_sampling = params["sampling_info"]["n_sample"]
n_copy = params["sampling_info"]["n_copy"]

# data path info
environment = "local"
if environment == "local":
    train_data_path = f"./{circuit_name}/data/{n_qubit}-qubit/{error_model}/p={100*error_rate}%/each_{each_n_shot}_shot/"
    target_state_path = f"./{circuit_name}/target_state/{state_class}/{n_qubit}-qubit/{error_model}/p={100*error_rate}%/"
    model_path = f"./{circuit_name}/model/{state_class}/{n_qubit}-qubit/{error_model}/p={100*error_rate}%/each_{each_n_shot}_shot/"
    train_log_path = f"./{circuit_name}/train_log/{n_qubit}-qubit/{error_model}/p={100*error_rate}%/each_{each_n_shot}_shot/"
if environment == "colab":
    from google.colab import drive
    drive.mount("/content/drive/")
    drive_path = "/content/drive/MyDrive/NQS4VD/GHZ"
    train_data_path = drive_path + f"/{circuit_name}/data/{n_qubit}-qubit/{error_model}/p={100*error_rate}%/each_{each_n_shot}_shot/"
    target_state_path = drive_path + f"/{circuit_name}/target_state/{state_class}/{n_qubit}-qubit/{error_model}/p={100*error_rate}%/"
    model_path = drive_path + f"/{circuit_name}/model/{state_class}/{n_qubit}-qubit/{error_model}/p={100*error_rate}%/each_{each_n_shot}_shot/"
    train_log_path = drive_path + f"/{circuit_name}/train_log/{n_qubit}-qubit/{error_model}/p={100*error_rate}%/each_{each_n_shot}_shot/"


def seed_settings(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    qucumber.set_random_seed(seed, cpu=True, gpu=False)

seed_settings(seed=seed)

## generate dataset

In [3]:
target_state = target_circuit.GHZ(n_qubit, state_class, error_model, error_rate)
utils.save_density_matrix(target_state, target_state_path)
meas_pattern_df, train_df = dataset.generate(target_state, n_qubit, error_model, each_n_shot)
dataset.save(meas_pattern_df, train_df, train_data_path)

0it [00:00, ?it/s]

measurement pattern 1/27 : ('X', 'X', 'X')



100%|██████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 3001.34it/s]
1it [00:00,  2.97it/s]

measurement pattern 2/27 : ('X', 'X', 'Y')



100%|██████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 2913.13it/s]
2it [00:00,  2.92it/s]

...


100%|██████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 3067.35it/s]
25it [00:08,  2.91it/s]

measurement pattern 26/27 : ('Z', 'Z', 'Y')



100%|██████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 2997.99it/s]
26it [00:09,  2.93it/s]

measurement pattern 27/27 : ('Z', 'Z', 'Z')



100%|██████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 3093.13it/s]
27it [00:09,  2.85it/s]


## load dataset

In [4]:
meas_result, target_rho, meas_label, meas_pattern = utils.load_dataset_DM(train_data_path, target_state_path)

## callback settings

In [5]:
n_on_epoch = 1
def save_model(nn_state, **kwargs):
    global n_on_epoch
    os.makedirs(model_path, exist_ok = True)
    nn_state.save(model_path + f"epoch{n_on_epoch}_model.pt")
    n_on_epoch = n_on_epoch + 1

def F_ideal_train(nn_state, **kwargs):
    save_model(nn_state)
    ideal_state = target_circuit.GHZ(n_qubit, state_class, "ideal", error_rate)
    train_state = utils.get_density_matrix(nn_state)
    F = np.trace(sqrtm(sqrtm(ideal_state)@train_state@sqrtm(ideal_state)))
    
    return (F.real)**2

def F_noisy_train(nn_state, **kwargs):
    noisy_state = target_circuit.GHZ(n_qubit, state_class, error_model, error_rate)
    train_state = utils.get_density_matrix(nn_state)
    F = np.trace(sqrtm(sqrtm(noisy_state)@train_state@sqrtm(noisy_state)))
    
    return (F.real)**2
    
def F_ideal_mevec(nn_state, **kwargs):
    ideal_state = target_circuit.GHZ(n_qubit, state_class, "ideal", error_rate)
    train_state = utils.get_density_matrix(nn_state)
    max_eigen_state = utils.get_max_eigen_vector(train_state)
    F = max_eigen_state.T.conjugate()@ideal_state@max_eigen_state
    
    return F.real

def create_callback(nn_state):
    metric_dict = {
        "ideal_train": F_ideal_train,
        "noisy_train": F_noisy_train,
        "ideal_mevec": F_ideal_mevec,
        "KL_Divergence": ts.KL,
    }
    space = nn_state.generate_hilbert_space()
    callbacks = [
        MetricEvaluator(
            period,
            metric_dict,
            target = target_rho,
            bases = meas_pattern,
            verbose = True,
            space = space,
        )
    ]
    
    return callbacks

In [6]:
nn_state = DensityMatrix(num_visible = num_visible, num_hidden = num_hidden, num_aux = num_aux, unitary_dict = unitaries.create_dict(), gpu = use_gpu)
callbacks = create_callback(nn_state)

## train

In [ ]:
nn_state.fit(
    data = meas_result,
    input_bases = meas_label,
    epochs = epoch,
    pos_batch_size = pbs,
    neg_batch_size = nbs,
    lr = lr,
    k = n_gibbs_step,
    bases = meas_pattern,
    callbacks = callbacks,
    time = True,
    optimizer = torch.optim.Adadelta,
    scheduler = torch.optim.lr_scheduler.StepLR,
    scheduler_args = {"step_size": lr_drop_epoch, "gamma": lr_drop_factor},
)

Epoch: 1	ideal_train = 0.431050	noisy_train = 0.582207	ideal_mevec = 0.515121	KL_Divergence = 0.352396
Epoch: 2	ideal_train = 0.425227	noisy_train = 0.626822	ideal_mevec = 0.569028	KL_Divergence = 0.130827
Epoch: 3	ideal_train = 0.459158	noisy_train = 0.639415	ideal_mevec = 0.553979	KL_Divergence = 0.153176
Epoch: 4	ideal_train = 0.461429	noisy_train = 0.656081	ideal_mevec = 0.646839	KL_Divergence = 0.088146
Epoch: 5	ideal_train = 0.467399	noisy_train = 0.667897	ideal_mevec = 0.992896	KL_Divergence = 0.070607
Epoch: 6	ideal_train = 0.471788	noisy_train = 0.670245	ideal_mevec = 0.884132	KL_Divergence = 0.070946
Epoch: 7	ideal_train = 0.478334	noisy_train = 0.674618	ideal_mevec = 0.840206	KL_Divergence = 0.068404
Epoch: 8	ideal_train = 0.480399	noisy_train = 0.676550	ideal_mevec = 0.939291	KL_Divergence = 0.067738
Epoch: 9	ideal_train = 0.486001	noisy_train = 0.686975	ideal_mevec = 0.763833	KL_Divergence = 0.072758
Epoch: 10	ideal_train = 0.494400	noisy_train = 0.689414	ideal_mevec = 0.6

## save train log

In [ ]:
os.makedirs(train_log_path, exist_ok = True)
train_log_df = pd.DataFrame()
train_log_df["epoch"] = np.arange(1, epoch+1, period)
train_log_df["F_ideal_train"] = callbacks[0]["ideal_train"]
train_log_df["F_noisy_train"] = callbacks[0]["noisy_train"]
train_log_df["F_ideal_meve"] = callbacks[0]["ideal_mevec"]
train_log_df["KL_Divergence"] = callbacks[0]["KL_Divergence"]
train_log_df.to_csv(train_log_path + "train_log.csv", index=False)